Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.


**Motivations** - This notebook explains how to setup and run a Command. The Command is a fundamental construct of Azure Machine Learning. It can be used to run a task on a specified compute (either local or on the cloud). The Command accepts `environment` and `compute` to setup required infrastructure. You can define a `command` to run on this infrastructure with `inputs`.

## Import the required libraries

In [1]:
# Import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential
import mlflow

from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes


# Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

In [2]:
# load enter details of your AML workspace from .env file
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.environ.get("subscription_id")
resource_group = os.environ.get("resource_group")
workspace = os.environ.get("workspace")

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), 
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace
)

# 2. Configure and run the Command
In this section we will configure and run a standalone job using the `command` class. The `command` class can be used to run standalone jobs and can also be used as a function inside pipelines.

## 2.1 Configure the Command
The `command` allows user to configure the following key aspects.
- `code` - This is the path where the code to run the command is located
- `command` - This is the command that needs to be run
- `inputs` - This is the dictionary of inputs using name value pairs to the command. The key is a name for the input within the context of the job and the value is the input value. Inputs can be referenced in the `command` using the `${{inputs.<input_name>}}` expression. To use files or folders as inputs, we can use the `Input` class. The `Input` class supports three parameters:
    - `type` - The type of input. This can be a `uri_file` or `uri_folder`. The default is `uri_folder`.         
    - `path` - The path to the file or folder. These can be local or remote files or folders. For remote files - http/https, wasb are supported. 
        - Azure ML `data`/`dataset` or `datastore` are of type `uri_folder`. To use `data`/`dataset` as input, you can use registered dataset in the workspace using the format '<data_name>:<version>'. For e.g Input(type='uri_folder', path='my_dataset:1')
    - `mode` - 	Mode of how the data should be delivered to the compute target. Allowed values are `ro_mount`, `rw_mount` and `download`. Default is `ro_mount`
- `environment` - This is the environment needed for the command to run. Curated or custom environments from the workspace can be used. Or a custom environment can be created and used as well. Check out the [environment](../../../../assets/environment/environment.ipynb) notebook for more examples.
- `compute` - The compute on which the command will run. In this example we are using a compute called `cpu-cluster` present in the workspace. You can replace it any other compute in the workspace. You can run it on the local machine by using `local` for the compute. This will run the command on the local machine and all the run details and output of the job will be uploaded to the Azure ML workspace.
- `distribution` - Distribution configuration for distributed training scenarios. Azure Machine Learning supports PyTorch, TensorFlow, and MPI-based distributed training. The allowed values are `PyTorch`, `TensorFlow` or `Mpi`.
- `display_name` - The display name of the Job
- `description` - The description of the experiment

In [3]:
exp_name = "iris-example"
compute_name = "cpu-cluster"
env_name = "AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1"

code_dir = './src/train'
model_name = ""
model_output_dir = 'outputs/'
dataset_name = ""
artifact_path = "model"

In [4]:


inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path="./data/iris.csv"),
    "C": 0.8,
    "kernel": "rbf",
    "coef0": 0.1,
}

# create the command
job = command(
    code=code_dir,  # local path where the code is stored
    command="python train.py --iris-csv ${{inputs.input_data}} --C ${{inputs.C}} --kernel ${{inputs.kernel}} --coef0 ${{inputs.coef0}}",
    inputs=inputs,
    environment=env_name,
    compute=compute_name,
    display_name=exp_name,
    experiment_name=exp_name,
    description="Train a model on the Iris dataset",
)

# submit the command
returned_job = ml_client.create_or_update(job)

# print run id
runid=returned_job.id
print(f"Submitted job run with ID: {runid}")

# get a URL for the status of the job
ml_client.jobs.studio_url



Uploading iris.csv (< 1 MB): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.88k/3.88k [00:00<00:00, 407kB/s]


Submitted job run with ID: /subscriptions/f804f2da-c27b-45ac-bf80-16d4d331776d/resourceGroups/rg-mlopsv2clas-505prod/providers/Microsoft.MachineLearningServices/workspaces/mlw-mlopsv2clas-505prod/jobs/lemon_net_2wttv098sl


AttributeError: 'JobOperations' object has no attribute 'studio_url'

In [ ]:
import time
from IPython.display import clear_output
from termcolor import colored

isFinish = False
while not isFinish:
    status = ml_client.jobs.get(returned_job.name).status
    # if run is still running or starting, print status
    if status == "Running":
        clear_output(wait=True)
        print(colored(f"Run is in status: {status}", "yellow"))
        time.sleep(10)
    if status == "Starting":
        clear_output(wait=True)
        print(colored(f"Run is in status: {status}", "yellow", attrs=['bold']))
        time.sleep(10)
    elif status == "Finished":
        clear_output(wait=True)
        print(colored("Run is finished!", "green"))
        isFinish = True
    else:
        clear_output(wait=True)
        print(colored(f"Run is in status: {status}", "red"))
        isFinish = True


Run is in status: Completed


In [6]:
run = ml_client.jobs.get(returned_job.name)
run

Experiment,Name,Type,Status,Details Page
iris-example,lemon_net_2wttv098sl,command,Completed,Link to Azure Machine Learning studio


In [14]:
from azure.ai.ml.entities import  Model

# register the model

In [16]:

MODEL_PATH = "model"

model_name = 'sklearn-iris-model'

ml_client.models.create_or_update(
    Model(
        path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/{MODEL_PATH}",
        name=model_name,
        type=AssetTypes.MLFLOW_MODEL
    )
) 

Model({'job_name': 'lemon_net_2wttv098sl', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'sklearn-iris-model', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/f804f2da-c27b-45ac-bf80-16d4d331776d/resourceGroups/rg-mlopsv2clas-505prod/providers/Microsoft.MachineLearningServices/workspaces/mlw-mlopsv2clas-505prod/models/sklearn-iris-model/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alibina8/code/dstoolkit-mlops-v2/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f0db675b880>, 'serialize': <msrest.serialization.Serializer object at 0x7f0db675b7f0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/f804f2da-c27b-45ac-bf80-16d4d331776d/resourceGroups/rg-mlopsv2clas-505prod/workspaces/mlw-mlopsv2clas-505prod/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.lemon_net_2wttv098sl/model', 'datastore': None, 'utc_time_c